In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

# all RRLyrae even remotly close to M49

In [ ]:
# all the locations of the RR Lyrae stars in the M49 field 
df_m49 = pd.read_parquet("/sdf/group/rubin/shared/notebooks/mjuric/rfl/variable_objects/w_18_objectId_RR_Lyrae_all_positions"+".parquet")
plt.scatter(df_m49['_RAJ2000'], df_m49['_DEJ2000'], s=1, c='black', alpha=0.5)

# Load lightcurves with all info

In [ ]:
good_index =  np.array([70927777120911428, 72591613091643579, 74238475351621650, 72584053949202434, 74241224130691123, 74242667239702535, 74249126870515736, 74244522665574412,  75894133704622083, 75895508094156806 ])

# Set the directory where files are saved
input_dir = "/sdf/group/rubin/shared/notebooks/mjuric/rfl/variable_objects/test"

# List to collect DataFrames
lightcurves = []

for object_id in good_index:
    file_path = f"{input_dir}/w_18_objectId_{int(object_id)}.parquet"
    if os.path.exists(file_path):
        try:
            df = pd.read_parquet(file_path)
            lightcurves.append(df)
        except Exception as e:
            print(f"Failed to load {file_path}: {e}")
    else:
        print(f"File not found: {file_path}")

# Combine all into one DataFrame
all_lc_df = pd.concat(lightcurves, ignore_index=True)

# Load lightcurves, fit and show them 

In [ ]:
good_index =  np.array([70927777120911428, 72591613091643579, 74238475351621650, 72584053949202434, 74241224130691123, 74242667239702535, 74249126870515736, 74244522665574412,  75894133704622083, 75895508094156806 ])

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

band_colors = {
    "u": "blue",
    "g": "green",
    "r": "red",
    "i": "brown"
}

def sinusoid(phase, A, phi0, mean_mag):
    return mean_mag + A * np.sin(2 * np.pi * phase + phi0)

for object_id in good_index:
    base_path = "/sdf/group/rubin/shared/notebooks/mjuric/rfl/variable_objects/test"
    
    # Load main lightcurve table
    lc_path = f"{base_path}/w_18_objectId_{int(object_id)}.parquet"
    single_lc = pd.read_parquet(lc_path)
    
    # Apply flag filtering if needed
    flag_cols = [col for col in single_lc.columns if 'flag' in col]
    if flag_cols:
        single_lc = single_lc[~single_lc[flag_cols].any(axis=1)]

    period = None  # Initialize period

    fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
    global_ymin = np.inf
    global_ymax = -np.inf

    for band, group in single_lc.groupby("band"):
        try:
            # Load per-band lightcurve
            lc_band_path = f"{base_path}/w_18_objectId_lightcurve_{int(object_id)}_{band}.parquet"
            lc_df = pd.read_parquet(lc_band_path)

            # Load fit parameters
            fit_path = f"{base_path}/w_18_objectId_popt_{int(object_id)}_{band}.parquet"
            fit_df = pd.read_parquet(fit_path)
            A, phi0, mean_mag = fit_df.loc[0, ['A', 'phi0', 'mean_mag']]
            period = fit_df.loc[0, 'period']

            # Extract data
            time = lc_df["mjd"].values
            mag = lc_df["mag"].values
            mag_err = lc_df["magErr"].values
            phase = lc_df["phase"].values

            # Model values
            phase_model = np.linspace(0, 1, 500)
            mag_model_phase = sinusoid(phase_model, A, phi0, mean_mag)

            t_model = np.linspace(np.min(time), np.max(time), 1000)
            phase_t_model = (t_model % period) / period
            mag_model_time = sinusoid(phase_t_model, A, phi0, mean_mag)

            # Update y-limits
            combined_mags = np.concatenate([mag, mag_model_phase, mag_model_time])
            global_ymin = min(global_ymin, np.nanmin(combined_mags))
            global_ymax = max(global_ymax, np.nanmax(combined_mags))

            # Plot: time domain
            axes[0].errorbar(time, mag, yerr=mag_err, fmt='o', label=f"{band}-band",
                             alpha=0.6, color=band_colors.get(band, 'gray'), capsize=3)
            axes[0].plot(t_model, mag_model_time, '--',
                         label=f"Fit ({band})", color=band_colors.get(band, 'gray'))

            # Plot: phase-folded
            axes[1].errorbar(phase, mag, yerr=mag_err, fmt='o', label=f"{band}-band",
                             alpha=0.6, color=band_colors.get(band, 'gray'), capsize=3)
            axes[1].plot(phase_model, mag_model_phase, '--',
                         label=f"Fit ({band})", color=band_colors.get(band, 'gray'))

        except Exception as e:
            print(f"Failed for object {object_id}, band {band}: {e}")
            continue

    for ax in axes:
        ax.invert_yaxis()
        ax.legend()
        ax.grid(True)
        ax.set_ylabel("Magnitude")

    axes[0].set_xlabel("MJD")
    axes[0].set_title("Time Domain")

    axes[1].set_xlabel("Phase")
    axes[1].set_xlim(0, 1)
    axes[1].set_title("Phase Folded")

    if period is not None:
        plt.suptitle(f"diaObject ID {object_id} / Period = {period:.4f} days")
    else:
        plt.suptitle(f"diaObject ID {object_id} / Period unknown")

    pad = 0.1 * (global_ymax - global_ymin)
    axes[0].set_ylim(global_ymax + pad, global_ymin - pad)
    axes[1].set_ylim(global_ymax + pad, global_ymin - pad)

    plt.tight_layout()
    plt.subplots_adjust(top=0.88)
    plt.show()